In [1]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_BookCrossing'

In [2]:
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pickle
from sklearn import metrics

In [4]:
ratings=pd.read_csv(f'{DATAPATH}/BX-Book-Ratings.csv',encoding='latin-1',error_bad_lines=False,sep=";")
print (ratings.head())


   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


In [5]:
ratings.shape, ratings['User-ID'].nunique(),ratings['ISBN'].nunique()

((1149780, 3), 105283, 340556)

In [6]:
ratings.columns=['userId','itemId','rating']

In [7]:
tempdf=ratings[['userId','rating']].groupby('userId').count()
tempdf=tempdf.loc[tempdf['rating']>=20] 
tempdf.shape# Only 7369 users have more than 20 ratings

(7369, 1)

In [8]:
ratings['num_ratings_per_user_gt_20']=ratings['userId'].isin(tempdf.index)

In [9]:
ratings['num_ratings_per_user_gt_20'].value_counts()

True     888884
False    260896
Name: num_ratings_per_user_gt_20, dtype: int64

In [10]:
tempdf=ratings[['itemId','rating']].groupby('itemId').count()
tempdf=tempdf.loc[tempdf['rating']>=200] 
tempdf.shape# Only 195 items have more than 20 ratings

(195, 1)

In [11]:
ratings['num_ratings_per_item_gt_200']=ratings['itemId'].isin(tempdf.index)

In [12]:
ratings['num_ratings_per_item_gt_200'].value_counts()

False    1084639
True       65141
Name: num_ratings_per_item_gt_200, dtype: int64

In [13]:
# Keep only explicit ratings - implicit means he interacted with the book in some way
ratings=ratings.loc[ratings['rating']>0]

In [14]:
ratings.shape, ratings['userId'].nunique(),ratings['itemId'].nunique()

((433671, 5), 77805, 185973)

In [15]:
ratings['flag_clean_considered']=ratings['num_ratings_per_user_gt_20']&ratings['num_ratings_per_item_gt_200']

In [16]:
ratings['flag_clean_considered'].value_counts()

False    418260
True      15411
Name: flag_clean_considered, dtype: int64

In [17]:
items=pd.read_csv (f'{DATAPATH}/BX-Books.csv',encoding='latin-1',error_bad_lines=False,sep=";")
print(items.head())

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'


         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-S  \
0  http://images.amazon.com/images/P/0195153448.0...   
1  http://images.amazon.com/

/home/kirana/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
items.rename({'ISBN':'itemId'},axis=1,inplace=True)

In [19]:
items.head()

,itemId,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [20]:
users=pd.read_csv(f'{DATAPATH}/BX-Users.csv',encoding='latin-1',error_bad_lines=False,sep=";")
print (users.head())

   User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN


In [21]:
users.rename({'User-ID':'userId'},axis=1,inplace=True)

In [22]:
users.head()

,userId,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [23]:
ratings.shape

(433671, 6)

In [24]:
ratings=pd.merge(ratings,users,on='userId',how='left')

In [25]:
ratings.shape

(433671, 8)

In [26]:
items.columns

Index(['itemId', 'Book-Title', 'Book-Author', 'Year-Of-Publication',
       'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [27]:
ratings=pd.merge(ratings,items.iloc[:,:5],on='itemId',how='left')

In [28]:
ratings.shape

(433671, 12)

In [29]:
ratings.head()

,userId,itemId,rating,num_ratings_per_user_gt_20,num_ratings_per_item_gt_200,flag_clean_considered,Location,Age,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276726,0155061224,5,False,False,False,"seattle, washington, usa",NaN,Rites of Passage,Judith Rae,2001,Heinle
1,276729,052165615X,3,False,False,False,"rijeka, n/a, croatia",16.0,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
2,276729,0521795028,6,False,False,False,"rijeka, n/a, croatia",16.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press
3,276736,3257224281,8,False,False,False,"salzburg, salzburg, austria",NaN,NaN,NaN,NaN,NaN
4,276737,0600570967,6,False,False,False,"sydney, new south wales, australia",14.0,NaN,NaN,NaN,NaN


In [30]:
ratings.shape

(433671, 12)

In [31]:
dfratings=ratings.copy()

### CROSS-VALIDATION 

In [32]:
# normal method is to keep the first x% of the ratings for each user as training and the rest as validation
# we want to simulate cold start by applying above only to users who were acquired in older years except last 2 years

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
ratings['rating'].value_counts()

8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: rating, dtype: int64

In [36]:
users=dfratings['userId'].unique()

In [37]:
users=np.random.permutation(users)

In [38]:
normalfrac=0.9

In [39]:
users_normal=users[:np.int(np.round(len(users)*normalfrac))]

In [40]:
dfnormal=dfratings.loc[dfratings['userId'].isin(users_normal)]

In [41]:
dfsimulate=dfratings.loc[~dfratings['userId'].isin(users_normal)]

In [42]:
dfnormal.shape, dfsimulate.shape

((392057, 12), (41614, 12))

In [43]:
temptrain,tempvalid=train_test_split(dfnormal,test_size=0.1,train_size=0.9,random_state=11,shuffle=True)

In [44]:
temptrain['random_dstype']='train'
tempvalid['random_dstype']='test'

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [45]:
dfnormal=pd.concat([temptrain,tempvalid],axis=0)

In [46]:
dfnormal.shape

(392057, 13)

In [47]:
dfnormal['random_dstype'].value_counts()

train    352851
test      39206
Name: random_dstype, dtype: int64

In [48]:
dfsimulate.shape

(41614, 12)

In [49]:
simulateusers=dfsimulate['userId'].unique()

In [50]:
simulateusers=np.random.permutation(simulateusers)

In [51]:
simulateusers_train=simulateusers[:np.int(np.round(len(simulateusers)*normalfrac))]

In [52]:
dfsimulate_train=dfsimulate.loc[dfsimulate['userId'].isin(simulateusers_train)]
dfsimulate_test=dfsimulate.loc[~dfsimulate['userId'].isin(simulateusers_train)]

In [53]:
dfsimulate_train['random_dstype']='train'
dfsimulate_test['random_dstype']='test'

/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/kirana/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [54]:
dfsimulate=pd.concat([dfsimulate_train,dfsimulate_test],axis=0)

In [55]:
dfratings=pd.concat([dfnormal,dfsimulate],axis=0)

In [56]:
dftrain=dfratings.loc[dfratings['random_dstype']=='train']

In [57]:
dfvalid=dfratings.loc[dfratings['random_dstype']!='train']

In [58]:
dftrain.shape, dfvalid.shape

((390999, 13), (42672, 13))

In [59]:
dfvalid['random_dstype'].value_counts()

test    42672
Name: random_dstype, dtype: int64

In [60]:
dfvalid['random_dstype'].value_counts()

test    42672
Name: random_dstype, dtype: int64

In [61]:
dftrain.columns

Index(['userId', 'itemId', 'rating', 'num_ratings_per_user_gt_20',
       'num_ratings_per_item_gt_200', 'flag_clean_considered', 'Location',
       'Age', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'random_dstype'],
      dtype='object')

In [62]:
dftrain.isnull().sum(axis=0)

userId                              0
itemId                              0
rating                              0
num_ratings_per_user_gt_20          0
num_ratings_per_item_gt_200         0
flag_clean_considered               0
Location                            0
Age                            115646
Book-Title                      44936
Book-Author                     44937
Year-Of-Publication             44936
Publisher                       44938
random_dstype                       0
dtype: int64

In [63]:
pickle.dump([dftrain,dfvalid],open(f'{DATAPATH}/df.pkl','wb'))

In [65]:
dftrain['userId'].nunique(),dftrain['itemId'].nunique()

(72887, 173090)